# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a better model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
datafile_train="train.csv"
datafile_test="test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [3]:
# you have been given additional data files which contain information about patient's 
# medical and demographic details . we'll merge those details with train and test 
# files using a common unique identifier 
b


In [4]:
medical_hist='medical_history.csv'
demo_det='demographic_details.csv'

In [5]:
med=pd.read_csv(medical_hist)

In [6]:
dem=pd.read_csv(demo_det)

In [7]:
# both the datasets have patient id as unique identifier 

In [8]:
med.nunique()

PatientId       62299
Hipertension        2
Diabetes            2
Alcoholism          2
Handcap             5
dtype: int64

In [9]:
dem.nunique()

PatientId        62299
Gender               2
Age                104
Neighbourhood       81
Scholarship          2
dtype: int64

In [10]:
# note that this patient info is for both train and test

In [64]:
comb=pd.merge(med,dem,how='outer',on='PatientId')

In [65]:
comb.head()

,PatientId,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Neighbourhood,Scholarship
0,2.987250e+13,1,0,0,0,F,62,JARDIM DA PENHA,0
1,5.589978e+14,0,0,0,0,M,56,JARDIM DA PENHA,0
2,4.262962e+12,0,0,0,0,F,62,MATA DA PRAIA,0
3,8.679512e+11,0,0,0,0,F,8,PONTAL DE CAMBURI,0
4,8.841186e+12,1,1,0,0,F,56,JARDIM DA PENHA,0


In [81]:
comb1 = comb[comb["Gender"]=='M']
comb2 = comb[comb["Gender"]=='F']
comb1 = np.mean(comb1["Age"]) # average
comb2 = np.mean(comb2["Age"])

In [82]:
comb1

33.32750640363097

In [83]:
comb2

38.58165609549019

In [62]:
round(len(comb1)/len(comb)*100)

7

In [13]:
bd_train.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No
1,5.589978e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No
2,8.679512e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No
3,8.841186e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No
4,9.598513e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No


In [14]:
train=pd.merge(bd_train,comb,how='left',on='PatientId')
test=pd.merge(bd_test,comb,how='left',on='PatientId')

In [15]:
train.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Neighbourhood,Scholarship
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No,1,0,0,0,F,62,JARDIM DA PENHA,0
1,5.589978e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No,0,0,0,0,M,56,JARDIM DA PENHA,0
2,8.679512e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No,0,0,0,0,F,8,PONTAL DE CAMBURI,0
3,8.841186e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No,1,1,0,0,F,56,JARDIM DA PENHA,0
4,9.598513e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No,1,0,0,0,F,76,REPÚBLICA,0


In [16]:
# we are going to drop columns 'PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'

# you should try making features out of date columns to improve performance of your model
# try the difference in dates , cyclic features from date components 

In [17]:
drop_cols=['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay']

In [18]:
train.drop(drop_cols,1,inplace=True)
test.drop(drop_cols,1,inplace=True)

In [19]:
train['data']='train'
test['data']='test'

all_data=pd.concat([train,test],0,sort=False)

In [20]:
all_data['gender_f']=(all_data['Gender']=='F').astype(int)
del all_data['Gender']

In [21]:
all_data.shape

(110344, 11)

In [22]:
k=all_data['Neighbourhood'].value_counts()
cats=k[k>2000].index

# you can try slightly loweer cutoff to include dummy vars for more neighbourhoods 
# check if that improves your model 

In [23]:
for cat in cats:
    name='Neighbourhood_'+cat
    all_data[name]=(all_data['Neighbourhood']==cat).astype(int)
del all_data['Neighbourhood']

In [24]:
all_data.shape

(110344, 31)

In [25]:
all_data.head()

,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Age,Scholarship,data,gender_f,...,Neighbourhood_SANTO ANDRÉ,Neighbourhood_CARATOÍRA,Neighbourhood_JABOUR,Neighbourhood_SÃO PEDRO,Neighbourhood_ILHA DO PRÍNCIPE,Neighbourhood_NOVA PALESTINA,Neighbourhood_DA PENHA,Neighbourhood_ANDORINHAS,Neighbourhood_ROMÃO,Neighbourhood_GURIGICA
0,0,No,1,0,0,0,62,0,train,1,...,0,0,0,0,0,0,0,0,0,0
1,0,No,0,0,0,0,56,0,train,0,...,0,0,0,0,0,0,0,0,0,0
2,0,No,0,0,0,0,8,0,train,1,...,0,0,0,0,0,0,0,0,0,0
3,0,No,1,1,0,0,56,0,train,1,...,0,0,0,0,0,0,0,0,0,0
4,0,No,1,0,0,0,76,0,train,1,...,0,0,0,0,0,0,0,0,0,0


In [26]:
target='No-show'

In [27]:
x_train=all_data.drop([target,'data'],1)[all_data['data']=='train']
y_train=all_data[target][all_data['data']=='train']
x_test=all_data.drop([target,'data'],1)[all_data['data']=='test']

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

C:\Users\KUMAR\Anaconda3_1\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\KUMAR\Anaconda3_1\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=T

In [29]:
params={'penalty':['l1','l2'],'class_weight':['balanced',None],
        'C':np.linspace(0.01,100,100)}

In [30]:
model=LogisticRegression()

# tune more complex algos for improving performance

In [31]:
rs=RandomizedSearchCV(model,param_distributions=params,n_iter=10,
                      scoring='roc_auc',cv=10,n_jobs=-1,verbose=20)

In [32]:
rs.fit(x_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


C:\Users\KUMAR\Anaconda3_1\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\KUMAR\Anaconda3_1\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to re

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='warn', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='warn', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributio...
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 8.283e+01, 8.384e+01,
       8.485e+01, 8.586e+01, 8.687e+01, 8.788e+01, 8.889e+01, 8.990e+01,
       9.091e+01, 9.192e+01, 9.293e+01, 9.394e+01, 9.495e+01, 9.596e+01,
       9.697e+

In [33]:
submissions=pd.DataFrame({'Junk':rs.predict_proba(x_test)[:,1]})
submissions.to_csv('submission.csv',index=False)